In [1]:
#!pip install datasets
from datasets import load_dataset, Dataset, DatasetDict
dataset_train=load_dataset("yelp_review_full",split='train')
dataset_test=load_dataset("yelp_review_full",split='test')

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
#Da qui distil-bert (è modello con meno parametri) per eseguire classification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model_name = 'distilbert-base-uncased'

distilbert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5) #num_labels
print(distilbert)


2024-05-02 12:58:57.046122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 12:58:57.046228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 12:58:57.174350: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
#upload del tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#I dati devono essere raccolti in un dizionario nella forma dataset per l'api
#Il load di dataset ti restituisce già un dizionario, ma io ho bisogno di split per validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val  = train_test_split(dataset_train['text'], dataset_train['label'], test_size=0.2, random_state=1)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
#Prepariamo dati per il training (serve lista di dizionari per tokenization)
train_data = [{'text': txt, 'label': lbl} for txt, lbl in zip(X_train, y_train)]
val_data = [{'text': txt, 'label': lbl} for txt, lbl in zip(X_val, y_val)]
train_data = Dataset.from_list(train_data)
val_data = Dataset.from_list(val_data)


In [5]:
data = DatasetDict()
data['train'] = train_data
data['validation'] = val_data
data['test'] = dataset_test#Già un dizionario nell


def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/520000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
print("vocabulary size: ", len(tokenizer.vocab)) 
#Con distilbert poche parole (30522)

vocabulary size:  30522


In [7]:
#La prossima funzione può prendere in ingresso fino a 109 input. Io setto come visto a lezione e qualcosina sulla gestione
#della gpu, ma si possono gestire anche soluzioni di early stopping al training e/o salvare il miglior modello su evaluation
#invece dell'ultimo trainato (questo è da provare in futuro per paragone performance). Per batch_size metto 16 come a lezione
#l'input ha come default 8
training_args = TrainingArguments(
    "trainer_distilbert", 
     per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    save_strategy='epoch'
)

In [8]:
trainer = Trainer(
    model=distilbert, 
    args=training_args, 
    train_dataset=tokenized_data['train'], 
    eval_dataset=tokenized_data['validation']
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
import wandb
wandb.login(key='5cb812ffc70e9a5ffc5dde2dd536729e61793ede')
trainer.train()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lucaquattro (prova_99). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240502_130520-ou5f4tut
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run wise-feather-14
wandb: ⭐️ View project at https://wandb.ai/prova_99/huggingface
wandb: 🚀 View run at https://wandb.ai/prova_99/huggingface/runs/ou5f4tut


Step,Training Loss
500,1.023100
1000,0.901000
1500,0.865000
2000,0.844200
2500,0.829900
3000,0.804000
3500,0.795400
4000,0.795600
4500,0.788200
5000,0.783700


TrainOutput(global_step=32500, training_loss=0.7084525259164663, metrics={'train_runtime': 28134.5972, 'train_samples_per_second': 36.965, 'train_steps_per_second': 1.155, 'total_flos': 1.377734651904e+17, 'train_loss': 0.7084525259164663, 'epoch': 2.0})

In [10]:
trainer.save_model(output_dir='/kaggle/working/')

In [11]:
trainer.evaluate()

{'eval_loss': 0.6991010308265686,
 'eval_runtime': 1273.8747,
 'eval_samples_per_second': 102.051,
 'eval_steps_per_second': 3.189,
 'epoch': 2.0}

In [12]:
testing_data = [{'text': txt, 'label': lbl} for txt, lbl in zip(dataset_test['text'][:256],dataset_test['label'][:256])]
testing_data= Dataset.from_list(testing_data)
data_test=DatasetDict()
data_test['test'] =  testing_data
tokenized_test = data_test.map(tokenize_function, batched=True)
preds = trainer.predict(tokenized_test['test'])

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

In [13]:
#Predizioni del test set
preds = trainer.predict(tokenized_data['test'])

In [14]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
y_pred = torch.argmax(torch.tensor(preds.predictions), dim=1).numpy()
print(confusion_matrix(dataset_test['label'], y_pred))
print(classification_report(dataset_test['label'], y_pred))

[[8040 1780  127   17   36]
 [1816 6523 1561   75   25]
 [ 188 1744 6328 1581  159]
 [  37   88 1480 6111 2284]
 [  49   19  148 2086 7698]]
              precision    recall  f1-score   support

           0       0.79      0.80      0.80     10000
           1       0.64      0.65      0.65     10000
           2       0.66      0.63      0.64     10000
           3       0.62      0.61      0.62     10000
           4       0.75      0.77      0.76     10000

    accuracy                           0.69     50000
   macro avg       0.69      0.69      0.69     50000
weighted avg       0.69      0.69      0.69     50000

